# Title
[]()

In [1]:

import sys
sys.path.append(r"/home/silvhua/custom_python")
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/')
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/app')
from silvhua import *
from chat_functions import *
from langchain.agents import Tool
from data_functions import parse_json_string

In [2]:
from app.ghl_requests import *
from dotenv import load_dotenv
load_dotenv(dotenv_path='../src/.env')

True

In [3]:
path = '/home/silvhua/repositories/GHL-chat/src/app/private'
contacts = load_json('contacts.json', path)
contactId = contacts['me_fb']

In [4]:
system_message_dict = dict()
conversation_dict = dict()
reply_dict = dict()
conversation_id = 1
question_id = 1
reply_dict[conversation_id] = dict()
inbound_dict = dict()

In [5]:
from app.ghl_requests import *
from dotenv import load_dotenv
load_dotenv(dotenv_path='../src/.env')
response = refresh_token()

Tokens retrieved from S3 for CoachMcloone.
Tokens saved to S3 CoachMcloone.


# Iteration 1

In [12]:
def chat_with_chatbot_test(user_input, agent_info):
    start_time = time()
    print(f'Chat history length: {len(agent_info["chat_history"])}')
    print(f'\nChat history:')
    for item in agent_info["chat_history"]:
        print(f'**{item.type.upper()}**: {item.content}')
    chat_history = agent_info['chat_history']
    last_message = chat_history[-1].content
    manychat_outbound_message_substrings = [
        "I'll be in touch as soon as I'm online next!",
        "I'll give you a personal message here shortly."
    ]
    previous_message_type = chat_history[-2].type
    last_message_type = chat_history[-1].type
    past_outbound_messages = [item.content for item in chat_history if item.type.lower() == 'ai']
    print(f'Past outbound messages: {[item for item in past_outbound_messages]}')
    if (last_message == user_input): ## Check that the current user_input is the most recent message        
        # If the last message is also Inbound, then join all inbound messages together and delete them from chat history
        if previous_message_type == 'human': 
            last_inbound_messages_list = []
            for item in chat_history[-1:0:-1]:
                if item.type.lower() == 'human':
                    last_inbound_messages_list.append(item.content)
                    truncated_history = chat_history[:-1]
                else:
                    break
            last_inbound_messages = '\n\n'.join(reversed(last_inbound_messages_list))
            user_input = last_inbound_messages
            print(f'Joining previous messages as full user input: {user_input}')
            chat_history = truncated_history
        else:
            chat_history = chat_history[:-1]

        result = agent_info['agent_executor']({
                "input": user_input,
                "chat_history": chat_history
            })  
        print(f'Agent response time: {time() - start_time} seconds')

    elif (last_message_type != 'human') & (any(substring in last_message for substring in manychat_outbound_message_substrings)):
        result = agent_info['agent_executor']({
                "input": user_input,
                "chat_history": chat_history
            })  
        print(f'Agent response time: {time() - start_time} seconds')
    else:
        result = dict()
        result['output'] = '{"response": "Abort Lambda function", "alert_human": false}'
    # Check that the generated response is not similar to a previously sent outbound message.
    for item in past_outbound_messages:
        if item in result['output']:
            result['output'] = '{"response": "[AI response similar to previous outbound message.]", "alert_human": true}'
            break
    return result

def test_chatbot(contactId, InboundMessage, tools):
    system_message_dict[conversation_id] = create_system_message('CoachMcloone')
    conversation_dict[conversation_id] = create_chatbot(
        contactId, system_message_dict[conversation_id], tools=tools,
        # model='gpt-4-32k'
        )

    reply_dict[conversation_id][question_id] = chat_with_chatbot_test(
        InboundMessage, conversation_dict[conversation_id]
    )
    chatbot_response = parse_json_string(reply_dict[conversation_id][question_id]["output"])

    return chatbot_response


tools = [
    Tool(
        name=f"placeholder_function",
        func=placeholder_function,
        description=f"Do not invoke this function.",
    )
]
inbound_id = 1
inbound_dict[inbound_id] = 'can we do a call?'
conversation_id = 1
reply_dict[conversation_id] = dict()
chatbot_response = test_chatbot(contactId, inbound_dict[inbound_id], tools)
chatbot_response

/home/silvhua/repositories/GHL-chat/src/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/silvhua/repositories/GHL-chat/src/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.openai_functions_agent.base.OpenAIFunctionsAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_openai_functions_agent instead.
  warn_deprecated(


Chat history length: 24

Chat history:
**HUMAN**: GET STARTED
**AI**: So tell me, what's the biggest obstacle holding you back from achieving this goal then?
**HUMAN**: 💪 Training
**AI**: Thanks Silvia 🙏 I'll give you a personal message here shortly.

Just quickly, what's your current height and weight right now?

PS: Yes this message is automated, (I know you probably guessed that) but if you reply here, I'll get back to you as soon as I'm online next 😊
**HUMAN**: 55 kg 150cm
**AI**: Thanks for providing your height and weight! So, if everything were to go to plan over the next 3-6 months, what results would you like to achieve exactly? 🤔
**HUMAN**: deadlift 300lbs
**AI**: Ah that's a great goal! Tell me, why is deadlifting 300lbs important to you, and how will that impact your life? 

I'd love to help you out the best that I can!  ❤
**HUMAN**: confidence
**AI**: Thank you for opening up to me! You are not alone there! 
**AI**: A lot of my clients used to struggle with confidence befo

{'response': 'Abort Lambda function', 'alert_human': False}

In [18]:
conversation_dict[conversation_id]['chat_history']

[HumanMessage(content='GET STARTED'),
 AIMessage(content="So tell me, what's the biggest obstacle holding you back from achieving this goal then?"),
 HumanMessage(content='💪 Training'),
 AIMessage(content="Thanks Silvia 🙏 I'll give you a personal message here shortly.\n\nJust quickly, what's your current height and weight right now?\n\nPS: Yes this message is automated, (I know you probably guessed that) but if you reply here, I'll get back to you as soon as I'm online next 😊"),
 HumanMessage(content='55 kg 150cm'),
 AIMessage(content='Thanks for providing your height and weight! So, if everything were to go to plan over the next 3-6 months, what results would you like to achieve exactly? 🤔'),
 HumanMessage(content='deadlift 300lbs'),
 AIMessage(content="Ah that's a great goal! Tell me, why is deadlifting 300lbs important to you, and how will that impact your life? \n\nI'd love to help you out the best that I can!  ❤"),
 HumanMessage(content='confidence'),
 AIMessage(content='Thank you

In [20]:
conversation_dict[conversation_id].add_user_message(inbound_dict[inbound_id])
conversation_dict[conversation_id]['chat_history']

AttributeError: 'dict' object has no attribute 'add_user_message'

## 1.1

In [22]:
def create_chatbot_test(contactId, system_message, tools, model="gpt-3.5-turbo-1106", verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        model=model, 
        model_kwargs={"response_format": {"type": "json_object"}} # https://platform.openai.com/docs/guides/text-generation/json-mode  # https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
        )
    message_history = DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=contactId,
        key={
            "SessionId": contactId,
            "type": 'ChatHistory',
            }
        )
    system_message = SystemMessage(
        content=(system_message),
        input_variables=['InboundMessage']
    )

    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    parser = SimpleJsonOutputParser(pydantic_object=Chatbot_Response)
    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor( # Example of creating a custom agent: https://python.langchain.com/docs/modules/agents/how_to/custom_agent
        agent=agent, tools=tools, 
        verbose=verbose, return_intermediate_steps=True,
        parser=parser  # Add the parser instance to the agent_executor
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'chat_history': message_history.messages,
        'history_instance': message_history
    }
    return agent_info

def chat_with_chatbot_test(user_input, agent_info):
    start_time = time()
    print(f'Chat history length: {len(agent_info["chat_history"])}')
    print(f'\nChat history:')
    for item in agent_info["chat_history"]:
        print(f'**{item.type.upper()}**: {item.content}')
    chat_history = agent_info['chat_history']
    last_message = chat_history[-1].content
    manychat_outbound_message_substrings = [
        "I'll be in touch as soon as I'm online next!",
        "I'll give you a personal message here shortly."
    ]
    previous_message_type = chat_history[-2].type
    last_message_type = chat_history[-1].type
    past_outbound_messages = [item.content for item in chat_history if item.type.lower() == 'ai']
    print(f'Past outbound messages: {[item for item in past_outbound_messages]}')
    if (last_message == user_input): ## Check that the current user_input is the most recent message        
        # If the last message is also Inbound, then join all inbound messages together and delete them from chat history
        if previous_message_type == 'human': 
            last_inbound_messages_list = []
            for item in chat_history[-1:0:-1]:
                if item.type.lower() == 'human':
                    last_inbound_messages_list.append(item.content)
                    truncated_history = chat_history[:-1]
                else:
                    break
            last_inbound_messages = '\n\n'.join(reversed(last_inbound_messages_list))
            user_input = last_inbound_messages
            print(f'Joining previous messages as full user input: {user_input}')
            chat_history = truncated_history
        else:
            chat_history = chat_history[:-1]

        result = agent_info['agent_executor']({
                "input": user_input,
                "chat_history": chat_history
            })  
        print(f'Agent response time: {time() - start_time} seconds')

    elif (last_message_type != 'human') & (any(substring in last_message for substring in manychat_outbound_message_substrings)):
        result = agent_info['agent_executor']({
                "input": user_input,
                "chat_history": chat_history
            })  
        print(f'Agent response time: {time() - start_time} seconds')
    else:
        result = dict()
        result['output'] = '{"response": "Abort Lambda function", "alert_human": false}'
    # Check that the generated response is not similar to a previously sent outbound message.
    for item in past_outbound_messages:
        if item in result['output']:
            result['output'] = '{"response": "[AI response similar to previous outbound message.]", "alert_human": true}'
            break
    return result

def test_chatbot(contactId, InboundMessage, tools):
    system_message_dict[conversation_id] = create_system_message('CoachMcloone')
    conversation_dict[conversation_id] = create_chatbot_test(
        contactId, system_message_dict[conversation_id], tools=tools,
        # model='gpt-4-32k'
        )
    conversation_dict[conversation_id]['history_instance'].add_user_message(InboundMessage)
    reply_dict[conversation_id][question_id] = chat_with_chatbot_test(
        InboundMessage, conversation_dict[conversation_id]
    )
    chatbot_response = parse_json_string(reply_dict[conversation_id][question_id]["output"])

    return chatbot_response


tools = [
    Tool(
        name=f"placeholder_function",
        func=placeholder_function,
        description=f"Do not invoke this function.",
    )
]
inbound_id = 1.1
inbound_dict[inbound_id] = 'can we do a call?'
conversation_id = 1
reply_dict[conversation_id] = dict()
chatbot_response = test_chatbot(contactId, inbound_dict[inbound_id], tools)
chatbot_response

Chat history length: 24

Chat history:
**HUMAN**: GET STARTED
**AI**: So tell me, what's the biggest obstacle holding you back from achieving this goal then?
**HUMAN**: 💪 Training
**AI**: Thanks Silvia 🙏 I'll give you a personal message here shortly.

Just quickly, what's your current height and weight right now?

PS: Yes this message is automated, (I know you probably guessed that) but if you reply here, I'll get back to you as soon as I'm online next 😊
**HUMAN**: 55 kg 150cm
**AI**: Thanks for providing your height and weight! So, if everything were to go to plan over the next 3-6 months, what results would you like to achieve exactly? 🤔
**HUMAN**: deadlift 300lbs
**AI**: Ah that's a great goal! Tell me, why is deadlifting 300lbs important to you, and how will that impact your life? 

I'd love to help you out the best that I can!  ❤
**HUMAN**: confidence
**AI**: Thank you for opening up to me! You are not alone there! 
**AI**: A lot of my clients used to struggle with confidence befo

{'response': 'Abort Lambda function', 'alert_human': False}

## 1.2

In [23]:
def create_chatbot_test(contactId, system_message, tools, model="gpt-3.5-turbo-1106", verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        model=model, 
        model_kwargs={"response_format": {"type": "json_object"}} # https://platform.openai.com/docs/guides/text-generation/json-mode  # https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
        )
    message_history = DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=contactId,
        key={
            "SessionId": contactId,
            "type": 'ChatHistory',
            }
        )
    system_message = SystemMessage(
        content=(system_message),
        input_variables=['InboundMessage']
    )

    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    parser = SimpleJsonOutputParser(pydantic_object=Chatbot_Response)
    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor( # Example of creating a custom agent: https://python.langchain.com/docs/modules/agents/how_to/custom_agent
        agent=agent, tools=tools, 
        verbose=verbose, return_intermediate_steps=True,
        parser=parser  # Add the parser instance to the agent_executor
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'chat_history': message_history.messages,
        'history_instance': message_history
    }
    return agent_info

def chat_with_chatbot_test(user_input, agent_info):
    start_time = time()
    print(f'Chat history length: {len(agent_info["chat_history"])}')
    print(f'\nChat history:')
    for item in agent_info["chat_history"]:
        print(f'**{item.type.upper()}**: {item.content}')
    chat_history = agent_info['chat_history']
    last_message = chat_history[-1].content
    manychat_outbound_message_substrings = [
        "I'll be in touch as soon as I'm online next!",
        "I'll give you a personal message here shortly."
    ]
    previous_message_type = chat_history[-2].type
    last_message_type = chat_history[-1].type
    past_outbound_messages = [item.content for item in chat_history if item.type.lower() == 'ai']
    print(f'Past outbound messages: {[item for item in past_outbound_messages]}')
    if (last_message == user_input): ## Check that the current user_input is the most recent message        
        # If the last message is also Inbound, then join all inbound messages together and delete them from chat history
        if previous_message_type == 'human': 
            last_inbound_messages_list = []
            for item in chat_history[-1:0:-1]:
                if item.type.lower() == 'human':
                    last_inbound_messages_list.append(item.content)
                    truncated_history = chat_history[:-1]
                else:
                    break
            last_inbound_messages = '\n\n'.join(reversed(last_inbound_messages_list))
            user_input = last_inbound_messages
            print(f'Joining previous messages as full user input: {user_input}')
            chat_history = truncated_history
        else:
            chat_history = chat_history[:-1]

        result = agent_info['agent_executor']({
                "input": user_input,
                "chat_history": chat_history
            })  
        print(f'Agent response time: {time() - start_time} seconds')

    elif (last_message_type != 'human') & (any(substring in last_message for substring in manychat_outbound_message_substrings)):
        result = agent_info['agent_executor']({
                "input": user_input,
                "chat_history": chat_history
            })  
        print(f'Agent response time: {time() - start_time} seconds')
    else:
        result = dict()
        result['output'] = '{"response": "Abort Lambda function", "alert_human": false}'
    # Check that the generated response is not similar to a previously sent outbound message.
    for item in past_outbound_messages:
        if item in result['output']:
            result['output'] = '{"response": "[AI response similar to previous outbound message.]", "alert_human": true}'
            break
    return result

def test_chatbot(contactId, InboundMessage, tools):
    system_message_dict[conversation_id] = create_system_message('CoachMcloone')
    conversation_dict[conversation_id] = create_chatbot_test(
        contactId, system_message_dict[conversation_id], tools=tools,
        # model='gpt-4-32k'
        )
    # Add InboundMessage to chat history since this is normally done by Lambda function but not in `chat_functions.py`
    conversation_dict[conversation_id]['history_instance'].add_user_message(InboundMessage)
    conversation_dict[conversation_id]['chat_history'] = conversation_dict[conversation_id]['history_instance'].messages
    reply_dict[conversation_id][question_id] = chat_with_chatbot_test(
        InboundMessage, conversation_dict[conversation_id]
    )
    chatbot_response = parse_json_string(reply_dict[conversation_id][question_id]["output"])

    return chatbot_response


tools = [
    Tool(
        name=f"placeholder_function",
        func=placeholder_function,
        description=f"Do not invoke this function.",
    )
]
inbound_id = 1.2
inbound_dict[inbound_id] = 'can we do a call?'
conversation_id = 1
reply_dict[conversation_id] = dict()
chatbot_response = test_chatbot(contactId, inbound_dict[inbound_id], tools)
chatbot_response

Chat history length: 26

Chat history:
**HUMAN**: GET STARTED
**AI**: So tell me, what's the biggest obstacle holding you back from achieving this goal then?
**HUMAN**: 💪 Training
**AI**: Thanks Silvia 🙏 I'll give you a personal message here shortly.

Just quickly, what's your current height and weight right now?

PS: Yes this message is automated, (I know you probably guessed that) but if you reply here, I'll get back to you as soon as I'm online next 😊
**HUMAN**: 55 kg 150cm
**AI**: Thanks for providing your height and weight! So, if everything were to go to plan over the next 3-6 months, what results would you like to achieve exactly? 🤔
**HUMAN**: deadlift 300lbs
**AI**: Ah that's a great goal! Tell me, why is deadlifting 300lbs important to you, and how will that impact your life? 

I'd love to help you out the best that I can!  ❤
**HUMAN**: confidence
**AI**: Thank you for opening up to me! You are not alone there! 
**AI**: A lot of my clients used to struggle with confidence befo

/home/silvhua/repositories/GHL-chat/src/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



{
    "response": "Ah I think with the right accountability and support, you could do amazing! Would it be helpful if we jumped on a call and mapped out a plan for you to get your training on track so you can finally achieve your deadlift goal, in a way that’s achievable and sustainable? ❤",
    "alert_human": false,
    "phone_number": null
}

> Finished chain.
Agent response time: 2.5626604557037354 seconds


{'response': '[AI response similar to previous outbound message.]',
 'alert_human': True}

## 1.3

In [25]:
def create_chatbot_test(contactId, system_message, tools, model="gpt-3.5-turbo-1106", verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        model=model, 
        model_kwargs={"response_format": {"type": "json_object"}} # https://platform.openai.com/docs/guides/text-generation/json-mode  # https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
        )
    message_history = DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=contactId,
        key={
            "SessionId": contactId,
            "type": 'ChatHistory',
            }
        )
    system_message = SystemMessage(
        content=(system_message),
        input_variables=['InboundMessage']
    )

    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    parser = SimpleJsonOutputParser(pydantic_object=Chatbot_Response)
    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor( # Example of creating a custom agent: https://python.langchain.com/docs/modules/agents/how_to/custom_agent
        agent=agent, tools=tools, 
        verbose=verbose, return_intermediate_steps=True,
        parser=parser  # Add the parser instance to the agent_executor
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'chat_history': message_history
    }
    return agent_info

def chat_with_chatbot_test(user_input, agent_info):
    start_time = time()
    chat_history = agent_info['chat_history'].messages
    print(f'Chat history length: {len(chat_history)}')
    print(f'\nChat history:')
    for item in chat_history:
        print(f'**{item.type.upper()}**: {item.content}')
    last_message = chat_history[-1].content
    manychat_outbound_message_substrings = [
        "I'll be in touch as soon as I'm online next!",
        "I'll give you a personal message here shortly."
    ]
    previous_message_type = chat_history[-2].type
    last_message_type = chat_history[-1].type
    past_outbound_messages = [item.content for item in chat_history if item.type.lower() == 'ai']
    print(f'Past outbound messages: {[item for item in past_outbound_messages]}')
    if (last_message == user_input): ## Check that the current user_input is the most recent message        
        # If the last message is also Inbound, then join all inbound messages together and delete them from chat history
        if previous_message_type == 'human': 
            last_inbound_messages_list = []
            for item in chat_history[-1:0:-1]:
                if item.type.lower() == 'human':
                    last_inbound_messages_list.append(item.content)
                    truncated_history = chat_history[:-1]
                else:
                    break
            last_inbound_messages = '\n\n'.join(reversed(last_inbound_messages_list))
            user_input = last_inbound_messages
            print(f'Joining previous messages as full user input: {user_input}')
            chat_history = truncated_history
        else:
            chat_history = chat_history[:-1]

        result = agent_info['agent_executor']({
                "input": user_input,
                "chat_history": chat_history
            })  
        print(f'Agent response time: {time() - start_time} seconds')

    elif (last_message_type != 'human') & (any(substring in last_message for substring in manychat_outbound_message_substrings)):
        result = agent_info['agent_executor']({
                "input": user_input,
                "chat_history": chat_history
            })  
        print(f'Agent response time: {time() - start_time} seconds')
    else:
        result = dict()
        result['output'] = '{"response": "Abort Lambda function", "alert_human": false}'
    # Check that the generated response is not similar to a previously sent outbound message.
    for item in past_outbound_messages:
        if item in result['output']:
            result['output'] = '{"response": "[AI response similar to previous outbound message.]", "alert_human": true}'
            break
    return result

def test_chatbot(contactId, InboundMessage, tools):
    system_message_dict[conversation_id] = create_system_message('CoachMcloone')
    conversation_dict[conversation_id] = create_chatbot_test(
        contactId, system_message_dict[conversation_id], tools=tools,
        # model='gpt-4-32k'
        )
    # Add InboundMessage to chat history since this is normally done by Lambda function but not in `chat_functions.py`
    conversation_dict[conversation_id]['chat_history'].add_user_message(InboundMessage)
    reply_dict[conversation_id][question_id] = chat_with_chatbot_test(
        InboundMessage, conversation_dict[conversation_id]
    )
    chatbot_response = parse_json_string(reply_dict[conversation_id][question_id]["output"])

    return chatbot_response


tools = [
    Tool(
        name=f"placeholder_function",
        func=placeholder_function,
        description=f"Do not invoke this function.",
    )
]
inbound_id = 1.3
inbound_dict[inbound_id] = 'can we do a call?'
conversation_id = 1
reply_dict[conversation_id] = dict()
chatbot_response = test_chatbot(contactId, inbound_dict[inbound_id], tools)
chatbot_response

Chat history length: 25

Chat history:
**HUMAN**: GET STARTED
**AI**: So tell me, what's the biggest obstacle holding you back from achieving this goal then?
**HUMAN**: 💪 Training
**AI**: Thanks Silvia 🙏 I'll give you a personal message here shortly.

Just quickly, what's your current height and weight right now?

PS: Yes this message is automated, (I know you probably guessed that) but if you reply here, I'll get back to you as soon as I'm online next 😊
**HUMAN**: 55 kg 150cm
**AI**: Thanks for providing your height and weight! So, if everything were to go to plan over the next 3-6 months, what results would you like to achieve exactly? 🤔
**HUMAN**: deadlift 300lbs
**AI**: Ah that's a great goal! Tell me, why is deadlifting 300lbs important to you, and how will that impact your life? 

I'd love to help you out the best that I can!  ❤
**HUMAN**: confidence
**AI**: Thank you for opening up to me! You are not alone there! 
**AI**: A lot of my clients used to struggle with confidence befo

{'response': '[AI response similar to previous outbound message.]',
 'alert_human': True}

## 1.4

In [26]:
def create_chatbot_test(contactId, system_message, tools, model="gpt-3.5-turbo-1106", verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        model=model, 
        model_kwargs={"response_format": {"type": "json_object"}} # https://platform.openai.com/docs/guides/text-generation/json-mode  # https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
        )
    message_history = DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=contactId,
        key={
            "SessionId": contactId,
            "type": 'ChatHistory',
            }
        )
    system_message = SystemMessage(
        content=(system_message),
        input_variables=['InboundMessage']
    )

    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    parser = SimpleJsonOutputParser(pydantic_object=Chatbot_Response)
    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor( # Example of creating a custom agent: https://python.langchain.com/docs/modules/agents/how_to/custom_agent
        agent=agent, tools=tools, 
        verbose=verbose, return_intermediate_steps=True,
        parser=parser  # Add the parser instance to the agent_executor
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'chat_history': message_history
    }
    return agent_info

def chat_with_chatbot_test(user_input, agent_info):
    start_time = time()
    chat_history = agent_info['chat_history'].messages
    print(f'Chat history length: {len(chat_history)}')
    print(f'\nChat history:')
    for item in chat_history:
        print(f'**{item.type.upper()}**: {item.content}')
    last_message = chat_history[-1].content
    manychat_outbound_message_substrings = [
        "I'll be in touch as soon as I'm online next!",
        "I'll give you a personal message here shortly."
    ]
    previous_message_type = chat_history[-2].type
    last_message_type = chat_history[-1].type
    past_outbound_messages = [item.content for item in chat_history if item.type.lower() == 'ai']
    print(f'Past outbound messages: {[item for item in past_outbound_messages]}')
    if (last_message == user_input): ## Check that the current user_input is the most recent message        
        # If the last message is also Inbound, then join all inbound messages together and delete them from chat history
        if previous_message_type == 'human': 
            last_inbound_messages_list = []
            for item in chat_history[-1:0:-1]:
                if item.type.lower() == 'human':
                    last_inbound_messages_list.append(item.content)
                    truncated_history = chat_history[:-1]
                else:
                    break
            last_inbound_messages = '\n\n'.join(reversed(last_inbound_messages_list))
            user_input = last_inbound_messages
            print(f'Joining previous messages as full user input: {user_input}')
            chat_history = truncated_history
        else:
            chat_history = chat_history[:-1]

        result = agent_info['agent_executor']({
                "input": user_input,
                "chat_history": chat_history
            })  
        print(f'Agent response time: {time() - start_time} seconds')

    elif (last_message_type != 'human') & (any(substring in last_message for substring in manychat_outbound_message_substrings)):
        result = agent_info['agent_executor']({
                "input": user_input,
                "chat_history": chat_history
            })  
        print(f'Agent response time: {time() - start_time} seconds')
    else:
        result = dict()
        result['output'] = '{"response": "Abort Lambda function", "alert_human": false}'
    # Check that the generated response is not similar to a previously sent outbound message.
    for item in past_outbound_messages:
        n_words = len(item.split())
        if (n_words > 3) & (item in result['output']):
            result['output'] = '{"response": "[AI response similar to previous outbound message.]", "alert_human": true}'
            break
    return result

def test_chatbot(contactId, InboundMessage, tools):
    system_message_dict[conversation_id] = create_system_message('CoachMcloone')
    conversation_dict[conversation_id] = create_chatbot_test(
        contactId, system_message_dict[conversation_id], tools=tools,
        # model='gpt-4-32k'
        )
    # Add InboundMessage to chat history since this is normally done by Lambda function but not in `chat_functions.py`
    conversation_dict[conversation_id]['chat_history'].add_user_message(InboundMessage)
    reply_dict[conversation_id][question_id] = chat_with_chatbot_test(
        InboundMessage, conversation_dict[conversation_id]
    )
    chatbot_response = parse_json_string(reply_dict[conversation_id][question_id]["output"])

    return chatbot_response


tools = [
    Tool(
        name=f"placeholder_function",
        func=placeholder_function,
        description=f"Do not invoke this function.",
    )
]
inbound_id = 1.4
inbound_dict[inbound_id] = 'can we do a call?'
conversation_id = 1
reply_dict[conversation_id] = dict()
chatbot_response = test_chatbot(contactId, inbound_dict[inbound_id], tools)
chatbot_response

Chat history length: 25

Chat history:
**HUMAN**: GET STARTED
**AI**: So tell me, what's the biggest obstacle holding you back from achieving this goal then?
**HUMAN**: 💪 Training
**AI**: Thanks Silvia 🙏 I'll give you a personal message here shortly.

Just quickly, what's your current height and weight right now?

PS: Yes this message is automated, (I know you probably guessed that) but if you reply here, I'll get back to you as soon as I'm online next 😊
**HUMAN**: 55 kg 150cm
**AI**: Thanks for providing your height and weight! So, if everything were to go to plan over the next 3-6 months, what results would you like to achieve exactly? 🤔
**HUMAN**: deadlift 300lbs
**AI**: Ah that's a great goal! Tell me, why is deadlifting 300lbs important to you, and how will that impact your life? 

I'd love to help you out the best that I can!  ❤
**HUMAN**: confidence
**AI**: Thank you for opening up to me! You are not alone there! 
**AI**: A lot of my clients used to struggle with confidence befo

{'response': '[AI response similar to previous outbound message.]',
 'alert_human': True}

# *End of Page*